In [ ]:
### Import Libraries.

library("Seurat")
library("dplyr")
library("ggplot2")
library("qs")S
library("speckle") 
library("SingleCellExperiment")
library("readr")
library("writexl")

In [ ]:
### Load 
setwd("/folder/")
Seurat <- qread("adata.qs")

In [ ]:
### Create Sample x Cluster Count Matrix and Matching Metadata.

make_counts_and_meta <- function(seurat_obj, cluster_col = "Cluster_Column", sample_col = "Sample_ID", status_col = "Status") {
  meta <- seurat_obj@meta.data %>%
    dplyr::select(all_of(c(sample_col, status_col, cluster_col))) %>%
    tibble::rownames_to_column(var = "cell_id")
  
  if (!all(c(sample_col, status_col, cluster_col) %in% colnames(seurat_obj@meta.data))) {
    stop("One or more required metadata columns are missing in Seurat object.")
  }
  
  meta <- meta %>%
    mutate(Sample_ID_Status = paste0(.data[[sample_col]], "_", .data[[status_col]]))
  
  counts_tbl <- meta %>%
    count(Sample_ID_Status, .data[[cluster_col]]) %>%
    tidyr::pivot_wider(names_from = !!rlang::sym(cluster_col),
                       values_from = n,
                       values_fill = 0)
  
  counts_mat <- counts_tbl %>%
    tibble::column_to_rownames(var = "Sample_ID_Status") %>%
    as.matrix()
  
  meta_df <- tibble::tibble(Sample_ID_Status = rownames(counts_mat)) %>%
    tidyr::separate(Sample_ID_Status, into = c(sample_col, status_col), sep = "_", remove = FALSE) %>%
    dplyr::mutate(!!status_col := factor(.data[[status_col]]))
  rownames(meta_df) <- meta_df$Sample_ID_Status
  
  list(counts = counts_mat, meta = meta_df)
}

In [ ]:
### Prepare Inputs for Cluster_Column Contrast.

prep <- make_counts_and_meta(SeuratObject_SC, cluster_col = "Cluster_Column",
                             sample_col = "Sample_ID", status_col = "Status")
count_mat <- prep$counts
meta_df   <- prep$meta

In [ ]:
### Filtered Desired Statuses for Contrast.

contrast_statuses <- c("Control", "sALS") ### Or C9ALS
keep_rows <- meta_df$Status %in% contrast_statuses
count_mat_contrast <- count_mat[keep_rows, , drop = FALSE]
meta_contrast      <- meta_df[keep_rows, , drop = FALSE]

In [ ]:
### Ensure Rownames Match and are in Same Order.

stopifnot(rownames(count_mat_contrast)[1:nrow(meta_contrast)] == rownames(meta_contrast))

In [ ]:
### Convert Counts to Single Cell Experiments.

sce <- SingleCellExperiment(assays = list(counts = t(count_mat_contrast)))
colData(sce)$sample <- rownames(meta_contrast)
colData(sce)$group <- meta_contrast$Status

In [ ]:
### Run Propeller (two transforms: logit and asin).
## Logit.

res_logit <- propeller(clusters = colnames(count_mat_contrast),
                       sample = rownames(meta_contrast),
                       group = meta_contrast$Status,
                       counts = count_mat_contrast,
                       transform = "logit",
                       robust = TRUE)

print("Propeller (logit) results:")
print(head(res_logit))

In [ ]:
### Run Propeller (two transforms: logit and asin).
## Asin

res_asin <- propeller(clusters = colnames(count_mat_contrast),
                      sample = rownames(meta_contrast),
                      group = meta_contrast$Status,
                      counts = count_mat_contrast,
                      transform = "asin",
                      robust = TRUE)

print("Propeller (asin) results:")
print(head(res_asin))

In [ ]:
### Save Results.

write.csv(res_logit, file = "propeller_results_logit.csv", row.names = TRUE)
write.csv(res_asin,  file = "propeller_results_asin.csv", row.names = TRUE)